In [9]:
import numpy as np
import pandas as pd
import pickle 
import torch
from model import BERT4Rec
import os
from utils import *
import yaml

In [14]:
EXP_N='' # 실험하고싶은 experiment number를 입력하세요

In [2]:
if torch.cuda.is_available() :
    cuda_aval = True
device = torch.device("cuda" if cuda_aval else "cpu")

In [3]:
if os.path.isfile('./data/new_user_movie_interaction.csv'):
    print("new_user_movie_interaction.csv 파일이 이미 존재합니다")
else:
    print("Start to make new_user_movie_interaction.csv (13minutes)")
    make_new_user_movie_interaction()
    print("new_user_movie_interaction.csv 생성 완료")
uim = pd.read_csv("./data/new_user_movie_interaction.csv")
user_id = uim["user"].to_numpy()

with open("./data/input_data.pickle", "rb") as fr : #user_seen_dict #preprocess.py에서 만들어짐
    raw_data = pickle.load(fr)

unique_sid = pd.read_csv("/opt/ml/input/data/train/pro_sg/unique_sid.txt", header=None)
show2id = dict((sid, i) for (i, sid) in enumerate(unique_sid))

item_id = unique_sid.to_numpy().reshape(6807)
item_id = np.insert(item_id, 0, 0)

new_user_movie_interaction.csv 파일이 이미 존재합니다


In [8]:
model = torch.load(f"./exp/experiment{EXP_N}/best.pth").to(device)
model.eval()

BERT4Rec(
  (item_emb): Embedding(6809, 60, padding_idx=0)
  (pos_emb): Embedding(50, 60)
  (dropout): Dropout(p=0.1, inplace=False)
  (emb_layernorm): LayerNorm((60,), eps=1e-06, elementwise_affine=True)
  (blocks): ModuleList(
    (0): BERT4RecBlock(
      (attention): MultiHeadAttention(
        (W_Q): Linear(in_features=60, out_features=60, bias=False)
        (W_K): Linear(in_features=60, out_features=60, bias=False)
        (W_V): Linear(in_features=60, out_features=60, bias=False)
        (W_O): Linear(in_features=60, out_features=60, bias=False)
        (attention): ScaledDotProductAttention(
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (dropout): Dropout(p=0.1, inplace=False)
        (layerNorm): LayerNorm((60,), eps=1e-06, elementwise_affine=True)
      )
      (pointwise_feedforward): PositionwiseFeedForward(
        (W_1): Linear(in_features=60, out_features=240, bias=True)
        (W_2): Linear(in_features=240, out_features=60, bias=True)
        (d

In [18]:
with open(f'./exp/experiment{EXP_N}/config.yaml') as f:
    config = yaml.load(f, Loader=yaml.FullLoader)
    print(config)

{'data_split': 'split_by_user', 'dropout_rate': 0.1, 'epochs': 2, 'eta_min': 0.0001, 'hidden_units': 60, 'lr': 0.001, 'lr_decay_step': 30, 'mask_prob': 0.15, 'max_seq_len': 50, 'name': 'experiment', 'num_heads': 1, 'num_layer': 2, 'patience': 10, 'seed': 42, 'tmax': 200, 'train_batch_size': 64, 'train_rating_path': '/opt/ml/input/data/train/train_ratings.csv', 'val_ratio': 0.2, 'valid_batch_size': 1024}
50


In [19]:
max_len = config['max_seq_len']
final = list()
cnt = 0
for key in raw_data.keys() :

    length = len(raw_data[key])
    if length < max_len : 
        dif = max_len-length
        data = [0]*dif + raw_data[key][-length:]
    else :
        data = raw_data[key][-max_len:]

    data = torch.LongTensor(data).unsqueeze(dim=0)
    result = model(data)[0]
    #t = result.detach().cpu().sum(axis=0)
    t = result[0].detach().cpu()
    t[data] = -np.inf
    top_k_idx = np.argpartition(t, -10)[-10:]
    rec_item_id = item_id[top_k_idx]
    user = user_id[key]


    for item in rec_item_id :
        final.append((user, item))

    cnt+=1

    if cnt%5000 == 0 :
        print(f"{cnt}/31360 complete")


5000/31360 complete
10000/31360 complete
15000/31360 complete
20000/31360 complete
25000/31360 complete
30000/31360 complete


In [21]:
if not os.path.isdir('./output'):
    os.makedirs('./output')

info = pd.DataFrame(final, columns=['user','item'])
info.to_csv(f"./output/submission_B4R_{EXP_N}.csv",index=False)
print("Inference Done!")

Inference Done!
